In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini")




In [3]:
from pydantic import BaseModel, Field


# Pydantic
class JsonLogicInterpretation(BaseModel):
    """A JSON logic interpretation."""
    rule: object = Field(description="The pure JSON logic rule expressed as a JSON object")
    examples: list[object] = Field(description="Three examples of data that we could run the JsonLogic rule on")
    variables: list[str] = Field(description="A list of variables referenced in the rule")


In [ ]:
import json


def fetch_json(path: str):
    with open(path) as json_data:
        return json.load(json_data)
  
def fetch_text(path: str):
    with open(path) as f:
        return f.read()


In [17]:
from langchain_core.prompts import PromptTemplate

prompt="""
I am looking to express laws as JsonLogic.

Here is some info on how JSON logic works:
---
{json_logic}
---

Please express the following law as JSON logic:
{law}

Provide your response as JSON in the following form:
// The pure JSON logic rule expressed as a JSON object
rule: object
// three examples of data that we could run the JsonLogic rule on
examples: object[]
// a list of variables referenced in the rule
variables: string[]

"""

prompt_template = PromptTemplate.from_template(prompt)



In [ ]:

json_logic = fetch_text("../markdown/json-logic.md")
law_object = fetch_json("../data/default/000000001.json")
law_text = law_object.get("text")
prompt = prompt_template.invoke({"json_logic": json_logic, "law": law_text})

structured_llm = llm.with_structured_output(JsonLogicInterpretation)

response = structured_llm.invoke(prompt)
print(response)

rule={'if': [{'and': [{'var': 'county_boundary_produced'}, {'var': 'water_area'}]}, 'jurisdiction', 'no_jurisdiction']} examples=[{'county_boundary_produced': True, 'water_area': True}, {'county_boundary_produced': True, 'water_area': False}, {'county_boundary_produced': False, 'water_area': True}] variables=['county_boundary_produced', 'water_area']
